In [4]:
# (48000, 28, 28) 크기인 train_input이
# (48000, 28, 28, 1) 크기인 train_scaled이 되었다.
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input.reshape(-1, 28, 28, 1) / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)


### 합성곱 신경망 만들기
- Sequential 클래스의 객체를 만들고 첫 번째 합성곱 층인 Conv2D를 추가
- 모델의 add() 메서드를 사용해 층을 하나씩 차례대로 추가

In [14]:
# 32개의 필터 사용, 커널 크기는 (3, 3), relu 함수, same padding
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=(28, 28, 1)))

In [15]:
# (28 28)크기에 세임 패딩을 적용했기 때문에 합성곱 층에서 출력된 특성 맵의 가로세로 크기는 입력과 동일
# (2,2) 풀링을 적용 -> 특성 맵의 크기는 줄어듬. (28, 28, 32) -> (14, 14, 32)
model.add(keras.layers.MaxPooling2D(2))

In [16]:
# 첫번째 합성곱-풀링층 다음에 두번째 합성곱-풀링층 추가. 둘다 거이 동일(필터의 개수만 64개로 늘런거 뻬고)
model.add(keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))

In [17]:
# 3차원 특성맵을 일렬로 펼침, 이유는 마지막의 10개의 뉴런을 가진 출력층에서 확률을 계산하기 때문.
# 특성맵을 일렬로 펼처서 바로 출력층에 전달하지 말고, 중간의 하나의 밀집 은닉층(hidden layer)를 하나 둠.
# 요약하면 Flatten class 다음, Dense 은닉층, 마지막으로 Dense 출력층 순서대로 구성
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu')) # 뉴런 100개
model.add(keras.layers.Dropout(0.4)) # 은닉층과 출력층 사이에 드롭아웃을 넣음, 과대적합 방지
model.add(keras.layers.Dense(10, activation='softmax')) 

In [18]:
# 합성곱 (1 -> 2, 32 -> 64)
# 특성맵의 크기는 첫번째, 2번째 풀링 거치면서 28->14->7. 최종 (7,7,64)
# 모델 파라미터 (32개 필터, 크기 (3,3), 길이는 1) = 3*3*1*32+32=320개, 2번째 3*3*32*64+64=18,946개
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 3136)              0         
                                                                 
 dense_2 (Dense)             (None, 100)              

In [24]:
!pip install pydot

In [28]:
# layer의 구성을 그림으로 표현, 근데 안떠
from keras.utils import plot_model
plot_model(model, to_file='model.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [29]:
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### 모델 컴파일과 훈련

In [33]:
# Adam optimizer 사용, ModelCheckpoint callback, EarlyStopping callback 함께 사용 -> 조기 종료 기법 구현
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5',save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
history = model.fit(train_scaled, train_target, epochs=20, validation_data=(val_scaled, val_target), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3599 - accuracy: 0.8710 - val_loss: 0.2876 - val_accuracy: 0.8923
Epoch 2/20
  15/1500 [..............................] - ETA: 11s - loss: 0.2440 - accuracy: 0.9021

/Users/daehyunkim/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1500/1500 [==============================] - 11s 7ms/step - loss: 0.3053 - accuracy: 0.8892 - val_loss: 0.2636 - val_accuracy: 0.9024
Epoch 3/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2723 - accuracy: 0.9010 - val_loss: 0.2491 - val_accuracy: 0.9064
Epoch 4/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2489 - accuracy: 0.9090 - val_loss: 0.2431 - val_accuracy: 0.9105
Epoch 5/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2250 - accuracy: 0.9175 - val_loss: 0.2370 - val_accuracy: 0.9118
Epoch 6/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2109 - accuracy: 0.9216 - val_loss: 0.2376 - val_accuracy: 0.9141
Epoch 7/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.1946 - accuracy: 0.9277 - val_loss: 0.2257 - val_accuracy: 0.9208
Epoch 8/20
1500/1500 [==============================] - 11s 7ms/step - loss: 0.1815 - accuracy: 0.9317 - val_loss: 0.2336 - val_accuracy: 0.9